# [DataChallenge Titanic](https://www.kaggle.com/competitions/titanic/overview)

Dans le cadre du [DataChallenge Titanic](https://www.kaggle.com/competitions/titanic/overview)  proposé dans le cours de Machine Learning chez YNOV, ce document servira de résumé descriptif pour [chacune des itérations soumises](https://www.kaggle.com/competitions/titanic/submissions) sur le site de Kaggle. 

Chaque itération décrite ci-après comprendra au moins un titre, une description, un algorithme de machine learning et le résultat associé. Dans certains cas on précisera quelles ont été les stratégies en détaillant par exemple la méthode utilisé pour l'optimisation, du code, etc..

# Première Itération : 
## Titre : Un petit tour sur une proposition formulée par ChatGPT

### Algorithme :
RandomForest 

### Description : 
Pour un premier jet, je voulais simplement tester la 'puissance' de ChatGPT, je lui ai donc soumis un prompt de manière à obtenir un premier résultat et avoir un aperçu du score que pouvait faire chatgpt en lui fournissant des consignes primaires :

```
I have a titanic dataset with PassengerID Pclass Name Sex Age SibSp Parch Ticket Fare Cabin Embarked and survived
I would like to make supervised learning by using a Model to train my algorithm. The target variable is  Survived others are caractértistiques variable how can i proceed to make a first simple submission
```

J'ai dû lui préciser certaines problématiques rencontrées comme les valeurs manquantes présentes dans certaines colonnes par exemple mais globalement je suis allé à l'essentiel pour obtenir de quoi soumettre une première version signé chatGPT.

### Code : 

```
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# Load the datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Handle missing values
train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
test_data['Age'].fillna(test_data['Age'].median(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].median(), inplace=True)
train_data['Embarked'].fillna(train_data['Embarked'].mode()[0], inplace=True)
test_data['Embarked'].fillna(test_data['Embarked'].mode()[0], inplace=True)

# Convert categorical variables into numeric
train_data['Sex'] = train_data['Sex'].map({'male': 0, 'female': 1})
test_data['Sex'] = test_data['Sex'].map({'male': 0, 'female': 1})
train_data['Embarked'] = train_data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
test_data['Embarked'] = test_data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

# Verify the conversion
print(train_data[['Sex', 'Embarked']].head())
print(test_data[['Sex', 'Embarked']].head())

# Select features
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X_train = train_data[features]
y_train = train_data['Survived']
X_test = test_data[features]

# Create the model
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)

# Train the model
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Prepare submission dataframe
submission = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],
    'Survived': predictions
})

# Save the submission to a CSV file
submission.to_csv('submission.csv', index=False)

```

### Score :
0.77751

### Conclusion : 
On va chercher à explorer diverses pistes en commençant par le choix des algorithmes et l'objectif sera dans un premier temps de faire une proposition personnel d'un model, dans un second temps d'optimiser mes recherches et donc 'in fine' le résultat du model pour battre ChatGPT et enfin chercher à se rapprocher le plus possible du meilleur score, soit 1.

# Deuxième Itération : 
## Titre : On démarre par la regression avec DecisionTreeRegressor

### Algorithme :
Pour cette première itération, nous choisissons d'utiliser l'arbre de décision. 

### Description :
Aborder un sujet de ML pour la première fois est un challenge. Les stratégies à déployer sont multiples et avant même l'écriture d'une seule ligne de code, le choix d'un algorithme plus ou moins complexe, les méthodes d'optimisation, etc.. on peut déjà se sentir dépassé par l'immensité des choix à faire à chaque itération. Pour éviter de se perdre au carrefour des chemins que l'on peut emprunter, nous choisissons de faire simple. Ainsi, nous allons démarrer par ce qu'on a le plus aborder en cours : La Regression. 

En s'inspirant d'un jupiter notebook vu en cours, nous avons adopter une sratégie très conservatrice, c'est-à-dire, une stratégie qui fait perdre beaucoup de données ici on a supprimer toutes les colonnes contenant des valeurs manquantes (Age, Cabin, Embarked) et celles n'étant pas des valeurs numériques (Name, Sex, Ticket). 

Nous avons ensuite séparer nos variables caractéristiques de notre variable cible pour entrainer notre model d'arbre de décision sur notre jeu de données: 

```
titanic_features = ['PassengerId', 'Pclass', 'SibSp', 'Parch', 'Fare']
X = df[titanic_features]
y = df.Survived
```

Puis nous allons séparer nos données d'entrainements pour obtenir un jeu d'entrainement et un jeu de validation. Ensuite, nous allons entrainer notre model `DecisionTreeRegressor` et procéder à une validation.

```
## Validation 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8) ## Possibilité de split le dataset différemment selon un paramètre à chercher pour les prochaines itérations

# instanciation du modèle
titanic_model = DecisionTreeRegressor(random_state=1)

# entraînement (fit)
titanic_model.fit(X_train, y_train)

# obtenir un score global (par défaut la metrique obtenu correspond à l'accuracy)
titanic_model.score(X_test, y_test)

## Le score obtenu est négatif, il devrait être compris entre 0 et 1. La méthode `.score` renvoie un coefficient de détermination R2 qui mesure la proportion de la variance des données qui est expliquée par le modèle. Un R2 de 1 indique un modèle parfait, tandis qu'un R2 de 0 indique que le modèle ne fait pas mieux que la moyenne des données.

Comment traduire ce résulat ? -0.4464646464646467
```

Nous décidons de poursuivre jusqu'à la soumission de notre première itération personnel quand bien même nous ne savons expliquer ce score négatif. 

### Score : 
Dans un premier temps un score de **0.0000** est obtenu tout simplement car la colonne `Survived` avait été laissé en float, une fois transformé en int64 nous obtenons **0.65071**

### Conclusion : 
Nous avons adopté ici une stratégie hyperconservatrice nous allons maintenant tenter d'améliorer notre résultat en travaillant sur l'encodage de certaines features qui ont été écartées comme par exemple les valeurs de la colonne `Sex`. Nous tenterons également d'optimiser nos paramètrès d'entrer de l'algorithme. À suivre ! 

# Troisième Itération :

## Titre : On poursuit la regression avec DecisionTreeRegressor(V2)

### Algorithme :
Pour cette nouvelle itération, nous choisissons de procéder à une deuxième version l'arbre de décision. 

### Description :
À partir de maintenant et ceci pour les futures itérations nous allons chercher à améliorer le précédent score réalisé (0.65071). Les leviers à actionner en ce sens sont les suivants :

- Nettoyage des données : Nous allons voir quelles stratégies nous pouvons adopter pour combler les valeurs manquantes (Imputation).
- Nous allons faire aussi du feature engineering : C'est-à-dire, réfléchir à comment pourrait-on exploiter les colones de notre datasets original pour extraire des informations différentes et peut-être plus fines permettant d'optimiser `in fine` notre score.
- Méthode d'optimisation : Nous allons mettre en place à minima un Grid Search pour tenter d'optimiser nos résultats.
- Utilisation d'autres modèles peut-être plus appropriés. 

Ici nous allons réfléchir dans un premier temps à remplacer les valeurs manquantes des colonnes ` Age`, `Embarked`, `Cabin`.

#### Valeurs Manquantes

##### Colonne `Age`

Nous avons remarqué qu'en mettant en perspective la colonne `Name` avec la colonne `Age`, qui contient 177 valeurs manquantes, on remarque qu'il y a des 'Titres' tels quel 'Mr', 'Mrs', 'Dr', 'Master', 'Miss', etc.. qui nous donne quelques indications sur l'âge que pourrait avoir nos personnes n'ayant pas leurs âges renseignés. Nous allons récupérer uniquement les lignes ayant des valeurs manquantes das la colonnes `Age` et voir comment remplacer de façon pertinente ces valeurs.

```
# Define a function to extract titles
def extract_title(name):
    return name.split(',')[1].split('.')[0].strip()

# Apply the function to create a 'Title' column
train_data['Title'] = train_data['Name'].apply(extract_title)

# Display
display(HTML(train_data['Title'].to_frame().to_html()))
```

```
# Count occurrences of each title within the train dataset
title_counts = train_data['Title'].value_counts()

print(title_counts)
```

Nous décidons alors de créer une colonne `Title`. De cette nouvelle colonne nous allons pouvoir déterminer la valeur median des âges selon les Titres propre à notre datasets. Nous ferons ensuite correspondre cette valeur médiane par Titre à chaque personne du même Titre n'ayant pas encore d'âge renseigné. 

```
selected_titles = ['Mr', 'Miss', 'Mrs', 'Master', 'Dr']
filtered_df = train_data[train_data['Title'].isin(selected_titles)]

# Calculate median age for each title
median_ages = filtered_df.groupby('Title')['Age'].median()

# Display median_ages
median_ages

# Replace NaN values in nan_age_rows['Age'] with median values based on Title
for index, row in nan_age_rows.iterrows():
    title = row['Title']
    median_age = median_ages[title]
    nan_age_rows.loc[index, 'Age'] = median_age
    train_data.loc[index, 'Age'] = median_age
```

##### Colonne `Embarked`

La majorité des personnes ont embarqués à S (Southampton), soit 684 personnes sur 891. Nous choisissons de remplacer les valeurs manquantes des deux lignes concernées par la valeur S.

```

# Replace NaN values in 'Embarked' with 'S'
train_data['Embarked'] = train_data['Embarked'].fillna('S')

```

##### Colonne `Cabin`

En ce qui concerne la colonne Cabin on remarque qu'il y a 77% environ de valeurs manquantes dans cette colonne. Analysons de plus prêt ces lignes.

En cherchant un plan du bateau du titanic on s'aperçoit que le paquebot était constitué de 10 ponts dont 7 avec cabines, allant du pont supérieure A au pont inférieur G. En regardant le plan et en comparant les résulats obtenus plus haut on s'aperçoit que la première lettre des cabines semblent correspondre avec les différents ponts présent sur le bateau.

Selon des informations récupérer sur la page wikipédia les [Passagers du Titanic](https://fr.wikipedia.org/wiki/Passagers_du_Titanic) "la troisième classe est destinée aux nombreux immigrants désireux de s'installer définitivement aux États-Unis.". Cet article nous informe également que "Le traitement de ces passagers varie selon la classe. Ainsi, les passagers de troisième classe sont soumis à de stricts contrôles sanitaires lors de l'embarquement, et sont totalement isolés des autres passagers, afin de faciliter les procédures d'arrivée à Ellis Island." 

**Par conséquent on peut imaginer que les ponts E, F et G sont ceux destinées aux personnes de la troisième classe**

**Que les ponts D, E et F sont ceux destinées aux personnes de la deuxième classe**

**Que les ponts A, B, C et D sont ceux destinées aux personnes de la premère classe**

En nous basant la colonne Pclass nous pouvons créer une colonne `Pont_de_Cabin`. Il y aurait à creuser plus en profondeur sur une répartition plus proche de la réalité mais nous choisissons pour cette itération de distribuer de façon aléatoire les potentielles ponts de cabine aux personnes selon leurs classe de la manière suivante : 

```
# Filter rows where 'Pclass' is equal to 1
pclass_1_rows = train_data[train_data['Pclass'] == 1]
#pclass_1_rows

# Further filter to include only rows where 'Cabin' is not NaN
pclass_1_rows_with_cabin = pclass_1_rows[pclass_1_rows['Cabin'].notna()]
pclass_1_rows_with_cabin

# Generate random choices for 'A', 'B', 'C' or 'D'
random_choices_class_1 = np.random.choice(['A', 'B', 'C', 'D'], size=len(pclass_1_rows))

# Assign these random choices to 'Pont_de_Cabin' for Pclass 2
train_data.loc[train_data['Pclass'] == 1, 'Pont_de_Cabin'] = random_choices_class_1
```

Nous nous sommes également rendu compte que selon les chiffres `paires` et `impaires` que l'on retrouve dans les cabines nous pouvions déterminer de quel côté du bateau se trouvaient les passagers : à babord pour les chiffres pairs à tribord pour les chiffres impairs. 

```
# Function to determine the side based on the cabin number
def determine_side(cabin):
    if pd.isna(cabin):
        return np.nan
    else:
        # Extract the numerical part of the cabin
        num_part = ''.join(filter(str.isdigit, cabin))
        if num_part:  # Check if there is a numerical part
            num = int(num_part)
            if num % 2 == 0:
                return 'Babord'
            else:
                return 'Tribord'
        else:
            return np.nan
```

On extrait l'information selon les chiffres présent dans la colonnes cabin

```
# Apply the function to create the 'Side' column
train_data['Side'] = train_data['Cabin'].apply(determine_side)

display(HTML(train_data['Side'].to_frame().to_html()))
```

Le problème c'est que nous ne savons pas pour le moment comment déterminer la répartition des chiffres pairs/impairs selon les persones de manière réaliste.
Nous décidons de ne pas aller plus loin dans le remplacement de valeurs manquantes pour cette itération. 

#### Encodage

Dans le but d'étoffer les variables caractéristiques sur lesquelles nous allons entraîner notre modèle nous décidons d'éncoder des variables nous utilisées jusqu'alors. Dans cette itération seule la colonne `Sex` est concernée.

##### Colonne `Sex`

```
# Encode 'Sex' column
train_data['Sex'] = train_data['Sex'].map({'male': 0, 'female': 1})
```

```
display(HTML(train_data['Sex'].to_frame().to_html()))
```
#### Validation 

Nous allons procéder à la réalisation de notre validation :

```
titanic_features = ['PassengerId', 'Pclass', 'SibSp', 'Parch', 'Fare', 'Sex', 'Age']
X = train_data[titanic_features]
y = train_data.Survived
```

##### La méthode Hold-out

```
## Validation 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8) ## Possibilité de split le dataset différemment selon un paramètre à chercher pour les prochaines itérations

# instanciation du modèle
titanic_model = DecisionTreeRegressor(random_state=1)

# entraînement (fit)
titanic_model.fit(X_train, y_train)

# obtenir un score global (par défaut la metrique obtenu correspond à l'accuracy)
titanic_model.score(X_test, y_test)

## Le score obtenu est négatif, il devrait être compris entre 0 et 1. La méthode `.score` renvoie un coefficient de détermination R2 qui mesure la proportion de la variance des données qui est expliquée par le modèle. Un R2 de 1 indique un modèle parfait, tandis qu'un R2 de 0 indique que le modèle ne fait pas mieux que la moyenne des données.

Comment traduire ce résulat ? -0.12671045429666106
```

Nous décidons de poursuivre jusqu'à la soumission de notre nouvelle itération personnel quand bien même nous ne savons expliquer ce score négatif. 

### Score : 
Nous obtenons un score de **0.74162**

### Conclusion : 
Nous avons ajouté simplement deux features à nos variables caractéristiques. Tout d'abord `Age` qui peut être exploiter maintenant qu'elle ne contient plus de valeur manquantes. Puis `Sex`qui a été encodé. Nous gagnons presque 0.1 point par rapport à notre précédente soumission. Il il y a d'autres voies encore à explorer que nous allons vous partagez dans notre prochaine itération.

# Quatrième Itération

## Titre : On poursuit la regression avec DecisionTreeRegressor(V3)

### Algorithme :
Pour cette nouvelle itération, nous choisissons de procéder à une troisième version l'arbre de décision. 

### Description :
À partir de maintenant et ceci pour les futures itérations nous allons chercher à améliorer le précédent score réalisé (0.74162). Les leviers à actionner en ce sens sont les suivants :

- Nettoyage des données : Nous allons voir quelle(s) stratégie(s) nous pouvons adopter pour combler les valeurs manquantes (Imputation).
- Encodage : Nous allons encoder les colonnes non exploitées jusqu'ici pour les ajouterparmi nos variables caractéristiques.
- Nous allons faire aussi du feature engineering : C'est-à-dire, réfléchir à comment pourrait-on exploiter les colonnes de notre dataset originel pour extraire des informations différentes et peut-être plus fines permettant d'optimiser `in fine` notre score.
- Méthode d'optimisation : Nous allons mettre en place à minima un Grid Search pour tenter d'optimiser nos résultats.
- Utilisation d'autres modèles peut-être plus appropriés. 

Ici nous allons procéder à l'encodage de nos variables `Title`, `Embarked`, `Pont_de_Cabin`, `Side`.

#### Encodage

##### Colonne `Title`

```
codes, uniques = pd.factorize(train_data['Title'])

train_data['Title_dummies'] = codes

train_data[['Title', 'Title_dummies']]
```

##### Colonne `Embarked`

```
codes, uniques = pd.factorize(train_data['Embarked'])
train_data['Embarked_dummies'] = codes
train_data[['Embarked', 'Embarked_dummies']]
```

##### Colonne `Pont_de_Cabin`

```
codes, uniques = pd.factorize(train_data['Pont_de_Cabin'])
train_data['Pont_de_Cabin_dummies'] = codes
train_data[['Pont_de_Cabin', 'Pont_de_Cabin_dummies']]
```

##### Colonne `Side`

```
codes, uniques = pd.factorize(train_data['Side'])
train_data['Side_dummies'] = codes
train_data[['Side', 'Side_dummies']]
```

Nous sommes conscients que dans ce contexte, les valeurs `NaN` présentes dans la colonne `Side`e sont également encoder et viennent donc créer un certain biais. Il faudra revenir sur ce point dans les futurs itération pour affiner le nettoyage en amont et l'encodage par la suite. Les conséquences de cette encodage sont la création de 4 nouvelles features `Title_dummies`, `Embarked_dummies`, `Pont_de_Cabin_dummies` et `Side_dummies`



#### Matrice de correlation

Avec un nombre croissant de variables caractéristiques il peut être intéressant de mettre en place une matrice de corrélation pour vérifier l'existence ou non de corrélation entre nos features.

```
# Compute the covariance matrix
covMatrix = np.cov(train_data[['Embarked_dummies', 'Title_dummies', 'Pclass', 'Pont_de_Cabin_dummies']].T, bias=False)

# Create a DataFrame for the covariance matrix to add labels
cov_df = pd.DataFrame(covMatrix, index=['Embarked_dummies', 'Title_dummies', 'Pclass', 'Pont_de_Cabin_dummies'], columns=['Embarked_dummies', 'Title_dummies', 'Pclass', 'Pont_de_Cabin_dummies'])
```

Nous avons sélectionné de manière intuitive 4 colonnes qui pourraient potentiellement présenter une correlation et nous allons afficher les résultats obtenus au sein d'une matrice en utilisant la bibliothèque seaborn : 

```
# Plot the covariance matrix using seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(cov_df, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Covariance Matrix')
plt.show()
```

#### Validation 

Nous allons procéder à la réalisation de notre validation en ajoutant nos nouvelles features pour entraîner notre modèle :

```
titanic_features = ['PassengerId', 'Pclass', 'SibSp', 'Parch', 'Fare', 'Sex', 'Age', 'Embarked_dummies', 'Title_dummies', 'Pont_de_Cabin_dummies']
X = train_data[titanic_features]
y = train_data.Survived
```

##### La méthode Hold-out

```
## Validation 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8) ## Possibilité de split le dataset différemment selon un paramètre à chercher pour les prochaines itérations

# instanciation du modèle
titanic_model = DecisionTreeRegressor(random_state=1)

# entraînement (fit)
titanic_model.fit(X_train, y_train)

# obtenir un score global (par défaut la metrique obtenu correspond à l'accuracy)
titanic_model.score(X_test, y_test)

## Le score obtenu est négatif, il devrait être compris entre 0 et 1. La méthode `.score` renvoie un coefficient de détermination R2 qui mesure la proportion de la variance des données qui est expliquée par le modèle. Un R2 de 1 indique un modèle parfait, tandis qu'un R2 de 0 indique que le modèle ne fait pas mieux que la moyenne des données.

Comment traduire ce résulat ? -0.027125717266562432
```

Nous décidons de poursuivre jusqu'à la soumission de notre nouvelle itération personnel quand bien même nous ne savons expliquer ce score négatif. 

### Score : 
Nous obtenons un score de **0.69377**. En relançant notre jupyter nptebook tout en récoltant les résultats de notre modèle dans un nouveau fichier de soumission nous obtenons aussi **0.56459** et **0.64354**.

### Conclusion : 
En encodant nos nouvelles colonnes nous avons choisis d'ajouter trois features à nos variables caractéristiques : `Title_dummies`, `Embarked_dummies` et `Pont_de_Cabin_dummies`. Nous perdons entre 0.15 et 0.5 points par rapport à notre précédente soumission. Il y a d'autres voies encore à explorer que nous allons vous partagez dans notre prochaine itération.

# Cinquième Itération

## Titre : Essai sur un autre modèle

### Algorithme : RandomForest

### Description : 

### Score : 
Nous avons obtenu un score de **0.77511** en prenant seulement 4 features (`Pclass`, `Sex`, `SibS`, `Parch`) encodées. 

### Conclusion : 
Bien que nous ayons utilisé que 4 features et sans faire un travail poussé en matière de nettoyage ou feature engineering nous obtenons un score supérieur à toutes les itérations préalablement réalisées. Le modèle random forest semble beaucoup plus approprié pour notre sujet. Dans les prochaines itérations, nous allons appliquer les mêmes stratégies d'imputation qu'appliquées lors des itérations réalisées avec le modèle de regression linéaire `DecisionTreeRegressor`. Nous espérons obtenir de meilleurs résultats encore. 

# Sixième Itération

## Titre : Essai sur un autre modèle (V2)

### Algorithme : RandomForest

### Description : 

### Score :
Nous avons obtenu un score de **0.78708** en rajoutant les features `Age` et `Fare` 

### Conclusion :
Bien que nous ayons utilisé que 6 features et en appliquant une méthode plus poussé pour remplacer les valeurs manquantes présentes dans nos colonnes `Age` et `Fare` nous gagnons 0.01 points. Ce n'est pas grand chose mais qui reste toujours une progression. Nous allons poursuivre sur ce Modèle Radom Forest en opérant du feature engineering. 